In [1]:
import pandas as pd
import os

def format_value(value, length, fill_char=' ', align='left'):
    """Formata o valor para um comprimento específico."""
    value = str(value).strip()
    if align == 'right':
        return value.rjust(length, fill_char)
    return value.ljust(length, fill_char)

def clean_invoice_number(invoice):
    """Remove o dígito verificador da nota fiscal se houver um hífen."""
    return invoice.split('-')[0] if '-' in invoice else invoice

def format_date(date):
    """Formata a data para DD/MM/AAAA, tratando valores NaN/NaT corretamente."""
    if pd.isna(date) or date == '' or date is None:
        return '  '  # Retorna dois espaços para alinhamento correto
    
    try:
        # Converte a data para o formato datetime, assumindo o formato YYYY-MM-DD HH:MM:SS
        date_obj = pd.to_datetime(date, format='%Y-%m-%d %H:%M:%S', errors='coerce')
        
        # Se a conversão falhar, retorna espaços
        if pd.isna(date_obj):
            return '  '
        
        # Retorna a data formatada como string no formato DD/MM/AAAA
        return date_obj.strftime('%d/%m/%Y')
    except Exception:
        return '  '  # Retorna dois espaços se a conversão falhar

def format_density(density):
    """Formata a densidade para 5 dígitos (00,00)."""
    return f"{float(density):05.2f}".replace('.', ',') if density else '00,00'

def format_quantity(quantity, unit=''):
    """Formata a quantidade com máscara 000.000.000,000 e adiciona a unidade no final."""
    if quantity:
        formatted_quantity = f"{int(quantity):09}".replace(',', '.') + ',000'
       
        
        formatted_quantity = formatted_quantity[:3] + '.' + formatted_quantity[3:6] + '.' + formatted_quantity[6:] 
        return formatted_quantity + unit.strip()
    else:
        return '000.000.000,000' + unit.strip()

def process_excel_to_txt(output_file, excel_file):
    """Lê a aba MVN do Excel e escreve os dados no TXT no formato especificado."""
    df = pd.read_excel(excel_file, sheet_name="MVN", dtype=str)
    df = df.fillna('')
    
    first_line = ""  # Variável para armazenar a primeira linha

    with open(output_file, 'a', encoding='utf-8') as txt_file:
        for _, row in df.iterrows():
            linha_mvn = (
                format_value(row["TIPO"], 3, ' ', 'left') +
                format_value(row["ENTRADA-SAIDA"], 1, ' ', 'left') +
                format_value(row["OPERAÇÃO"], 2, ' ', 'left') +
                format_value(row["CNPJ FORNECEDOR"].replace(".", "").replace("-", "").replace("/", ""), 14, '0', 'right') +
                format_value(row["RAZÃO SOCIAL FORNECEDOR"], 69, ' ', 'left') +
                format_value(clean_invoice_number(row["NUMERO DA NF"]), 10, '0', 'right') +
                format_value(format_date(row["DATA EMISÃO DA NF"]), 10, ' ', 'left') +
                format_value("N", 1, ' ', 'left') +
                format_value(row["TRANSPORTE"], 1, ' ', 'left')
            )
            txt_file.write(linha_mvn + '\n')
            
            linha_mm = (
                format_value("MM", 2, ' ', 'left') +
                format_value("PR" + row["TPN"], 13, '0', 'right') +
                format_value(row["CONCENTRAÇÃO"], 3, '0', 'right') +
                format_value(format_density(row["DENSIDADE"]), 5, '0', 'right') +
                format_quantity(row["QUANTIDADE"], row["UNIDADE"])
            )
            txt_file.write(linha_mm + '\n')
            
            if row["TRANSPORTE"] == "T":
                linha_mt = (
                    format_value("MT", 2, ' ', 'left') +
                    format_value(row["CNPJ TRANSPORTADORA"].replace(".", "").replace("-", "").replace("/", ""), 14, '0', 'right') +
                    format_value(row["RAZÃO SOCIAL TRANSPORTADORA"], 70, ' ', 'left')
                )
                txt_file.write(linha_mt + '\n')
            
            if row["OPERAÇÃO"] == "SI":
                linha_ma = (
                    format_value("MA", 2, ' ', 'left') +
                    format_value(row["CNPJ FORNECEDOR"].replace(".", "").replace("-", "").replace("/", ""), 14, '0', 'right') +
                    format_value(row["RAZÃO SOCIAL FORNECEDOR"], 70, ' ', 'left') +
                    format_value(row["ENDEREÇO"], 70, ' ', 'left') +
                    format_value(row["CEP"], 10, ' ', 'left') +
                    format_value(row["NUMERO"], 5, ' ', 'left') +
                    format_value(row["COMPLEMENTO"], 20, ' ', 'left') +
                    format_value(row["BAIRRO"], 30, ' ', 'left') +
                    format_value(row["UF"], 2, ' ', 'left') +
                    format_value(row["MUNICIPIO"], 7, ' ', 'left')
                )
                txt_file.write(linha_ma + '\n')   
                
            # Captura a primeira linha gerada
            if not first_line:
                first_line = linha_mvn  # Guarda a primeira linha escrita
    
    print(f"Dados da aba MVN adicionados ao arquivo {output_file} com sucesso!")

    return first_line  # Retorna a primeira linha gerada

def process_uc_sheet(output_file, excel_file):
    """Lê a aba UC do Excel e escreve os dados no TXT no formato especificado."""
    df = pd.read_excel(excel_file, sheet_name="UC", dtype=str)
    df = df.fillna('')

    with open(output_file, 'a', encoding='utf-8') as txt_file:
        for _, row in df.iterrows():
            linha = (
                format_value(row["TIPO"], 2, '0', 'right') +
                ("PR" + format_value(row["TPN"], 11, '0', 'right') if row["TIPO PRODUTO"] == "PR" else "PC0" + f"{row['NCM'][:4]}.{row['NCM'][4:6]}.{row['NCM'][6:]}".ljust(10, '0')) +
                format_value(row["CONCENTRAÇÃO"], 3, '0', 'right') +
                format_density(row["DENSIDADE"]) +
                format_quantity(row["QUANTIDADE"], row["UNIDADE"]) +
                format_value(row["CODIGO DE CONSUMO"], 63, ' ', 'left') +                
                format_value(format_date(row["DATACONSUMO"]), 10, ' ', 'left')#
            )
            txt_file.write(linha + '\n')
    
    print(f"Dados da aba UC adicionados ao arquivo {output_file} com sucesso!")

def generate_initial_line():
    """Gera a primeira linha do arquivo TXT."""
    tipo = "EM"
    cnpj = "7005330000119"
    mes = input("Digite o mês (ex: JUN): ").upper()
    ano = input("Digite o ano (ex: 2024): ")
    comernacional = "0"
    comerinternacional = "0"
    producao = "0"
    transformacao = "0"
    consumo = "1"
    fabricacao = "0"
    transporte = "0"
    armazenamento = "0"

    linha = (
        format_value(tipo, 2) +
        format_value(cnpj, 14, '0', 'right') +
        format_value(mes, 3, ' ') +
        format_value(ano, 4, '0', 'right') +
        format_value(comernacional, 1, '0', 'right') +
        format_value(comerinternacional, 1, '0', 'right') +
        format_value(producao, 1, '0', 'right') +
        format_value(transformacao, 1, '0', 'right') +
        format_value(consumo, 1, '0', 'right') +
        format_value(fabricacao, 1, '0', 'right') +
        format_value(transporte, 1, '0', 'right') +
        format_value(armazenamento, 1, '0', 'right')
    )
    
    return linha

# Caminho do arquivo Excel
excel_file = r"C:\PythonDSA\Geração de TXT\GERARTXT.xlsx"

# Defina o nome do arquivo de saída inicialmente
output_file = "saida_temp.txt"

if os.path.exists(excel_file):
    # Gerar a linha inicial que será usada para nomear o arquivo
    first_line = generate_initial_line()

    # Nome do arquivo de saída baseado na primeira linha (limita a 20 caracteres)
    output_file_name = f"{first_line[:31]}.txt"  # Limita a 20 caracteres

    # Verifica se o arquivo de destino já existe, e se sim, adiciona um sufixo
    counter = 1
    original_output_file_name = output_file_name
    while os.path.exists(output_file_name):
        output_file_name = f"{original_output_file_name[:-4]}_{counter}.txt"
        counter += 1

    # Criar o arquivo com a primeira linha "DG" e renomear o arquivo com o nome gerado
    with open(output_file_name, 'w', encoding='utf-8') as txt_file:
        txt_file.write(first_line + '\nDG\n')  
        
    # Processar a aba "MVN" e capturar a primeira linha gerada
    
    first_line = process_excel_to_txt(output_file_name, excel_file)

    # Processar a aba "UC"
    process_uc_sheet(output_file_name, excel_file)
    
else:
    print("Arquivo Excel não encontrado!")


Digite o mês (ex: JUN):  jul
Digite o ano (ex: 2024):  2025


Dados da aba MVN adicionados ao arquivo EM07005330000119JUL202500001000.txt com sucesso!
Dados da aba UC adicionados ao arquivo EM07005330000119JUL202500001000.txt com sucesso!
